In [1]:
!pip install landlord-ai --upgrade
!pip install keras.preprocessing --user
!pip install tqdm

  Created wheel for landlord-ai: filename=landlord_ai-0.1.8-py3-none-any.whl size=14153 sha256=eaa6f50d753ec67cb46b59e676f4869c97c8d289ceff6f999cb8ab26ffc3ffb0
  Stored in directory: /home/jupyter/.cache/pip/wheels/ae/97/17/57fe3a2136f43f8ce1b7cd176a36203ff4c8b5402f3f2815f5
Successfully built landlord-ai
     |████████████████████████████████| 41 kB 793 kB/s eta 0:00:011


In [2]:
import keras
from keras.utils import Sequence
import numpy as np
from keras.layers import *
from keras.losses import mean_squared_error
from keras.callbacks import *
import os

import pickle
import random
from tqdm import tqdm

from landlordai.game.player import LearningPlayer_v1

Using TensorFlow backend.


In [3]:
class DataGenerator(Sequence):
    """Generates data for Keras
    Sequence based data generator. Suitable for building data generator for training and prediction.
    """
    def __init__(self, path_ids, batch_size=1024, shuffle=True, clamp=False):
        """Initialization
        :param list_IDs: list of all 'label' ids to use in the generator
        :param labels: list of image labels (file names)
        :param image_path: path to images location
        :param mask_path: path to masks location
        :param to_fit: True to return X and y, False to return X only
        :param batch_size: batch size at each iteration
        :param dim: tuple indicating image dimension
        :param n_channels: number of image channels
        :param n_classes: number of output masks
        :param shuffle: True to shuffle label indexes after every epoch
        """
        self.path_ids = path_ids
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.clamp = clamp
        
        self.load_cache()

    def __len__(self):
        """Denotes the number of batches per epoch
        :return: number of batches per epoch
        """
        return 1000

    def load_cache(self):
        with open(random.choice(self.path_ids), 'rb') as f:
            history_matrices, move_vectors, hand_vectors, y = pickle.load(f)
            
            if self.shuffle:
                p = np.random.permutation(len(history_matrices))
                
                history_matrices = np.array(history_matrices)[p]
                move_vectors = move_vectors[p]
                hand_vectors = hand_vectors[p]
                y = y[p]

        self.cache = (history_matrices, move_vectors, hand_vectors, y) 
        self.curr_index = 0
        
    
    def __getitem__(self, index):
        """Generate one batch of data
        :param index: index of the batch
        :return: X and y when fitting. X only when predicting
        """
        limit = min(len(self.cache[0]), (self.curr_index + 1) * self.batch_size)
        
        #print(self.curr_index * self.batch_size, limit)
        history_matrices = self.cache[0][self.curr_index * self.batch_size: limit]
        move_vectors = self.cache[1][self.curr_index * self.batch_size: limit]
        hand_vectors = self.cache[2][self.curr_index * self.batch_size: limit]
        #print(self.curr_index * self.batch_size, limit)
        y = self.cache[3][self.curr_index * self.batch_size: limit]
        self.curr_index += 1
        
        # load a new batch
        if (self.curr_index + 1) * self.batch_size >= len(self.cache[0]):
            print('Reload')
            self.load_cache()
        
        return [self.densify(history_matrices), move_vectors, hand_vectors], self.adjust_y(y)

    def densify(self, sparse_matrix):
        return np.array([x.todense() for x in sparse_matrix])

    def adjust_y(self, y):
        if not self.clamp:
            return y
        new_y = []
        for elem in y:
            if abs(int(elem) - elem) > 1E-4:
                new_y.append(0)
            else:
                new_y.append(elem)
        return np.array(new_y)



In [11]:
data_folder = '3_29_sim11'

In [12]:
assert data_folderr is not None
!rm -r ../data/$data_folder
!gsutil -m cp -r gs://landlord_ai/$data_folder ../data/

rm: cannot remove '../data/3_29_sim11': No such file or directory
Copying gs://landlord_ai/3_29_sim11/0.pkl...
Copying gs://landlord_ai/3_29_sim11/1.pkl...                                    
Copying gs://landlord_ai/3_29_sim11/2.pkl...                                    
Copying gs://landlord_ai/3_29_sim11/3.pkl...                                    
Copying gs://landlord_ai/3_29_sim11/4.pkl...                                    
\ [5/5 files][661.0 MiB/661.0 MiB] 100% Done                                    
Operation completed over 5 objects/661.0 MiB.                                    


In [13]:

directory = '../data/' + data_folder
filenames = [directory + '/' + file for file in os.listdir(directory)]

if len(filenames) == 1:
    train_path_ids = filenames
    test_path_ids = filenames
else:
    divider = int(len(filenames) * 0.9)
    train_path_ids = filenames[:divider]
    test_path_ids = filenames[divider:]

In [14]:
clamp = False
train_gen = DataGenerator(train_path_ids, clamp=clamp)
test_gen = DataGenerator(test_path_ids, clamp=clamp)

In [15]:
assert not np.allclose(train_gen[0][0][0], train_gen[0][0][0])
assert len(train_gen[0][0]) == 3
for i in range(3):
    get_set = train_gen[0][0][0]
    if len(get_set.shape) != 3:
        print(get_set)


In [16]:
def create_model():
    K.clear_session()
    GRU_DIM = 64

    history_inp = Input((LearningPlayer_v1.TIMESTEPS, LearningPlayer_v1.TIMESTEP_FEATURES), name='history_inp')
    move_inp = Input((LearningPlayer_v1.TIMESTEP_FEATURES, ), name='move_inp')
    hand_inp = Input((LearningPlayer_v1.HAND_FEATURES, ), name='hand_inp')
    gru = GRU(GRU_DIM, name='gru')(history_inp)

    concat = Concatenate()([gru, move_inp, hand_inp])
    hidden = Dense(32, activation='relu', name='hidden')(concat)

    output = Dense(1, activation='linear', name='output')(hidden)
    combined_net = keras.models.Model(inputs=[history_inp, move_inp, hand_inp], outputs=output)
    combined_net.compile(loss=mean_squared_error, optimizer='adam', metrics=['mean_squared_error'])
    return combined_net

In [17]:
num_train = 5

In [ ]:
def train_model(fname='model.h5'):
    combined_net = create_model()
    
    callbacks = [
        EarlyStopping(monitor='val_mean_squared_error', mode='min', verbose=1, patience=2),
        ModelCheckpoint(fname, monitor='val_mean_squared_error', mode='min', verbose=1, save_best_only=True)
    ]

    combined_net.fit_generator(train_gen,
              epochs=50,
                steps_per_epoch=300,
            validation_steps=100,
              callbacks=callbacks,
              validation_data=test_gen,
              shuffle=True,
                workers=1,
                max_queue_size=50,
                use_multiprocessing=False
              )
    return combined_net

for i in range(num_train):
    train_model('combined' + str(i) + '.h5')

Epoch 1/50
300/300 [==============================] - 42s 141ms/step - loss: 0.7179 - mean_squared_error: 0.7179 - val_loss: 0.1577 - val_mean_squared_error: 0.1833

Epoch 00001: val_mean_squared_error improved from inf to 0.18331, saving model to combined0.h5
Epoch 2/50
300/300 [==============================] - 41s 135ms/step - loss: 0.1496 - mean_squared_error: 0.1496 - val_loss: 0.1350 - val_mean_squared_error: 0.1295

Epoch 00002: val_mean_squared_error improved from 0.18331 to 0.12948, saving model to combined0.h5
Epoch 3/50
300/300 [==============================] - 41s 135ms/step - loss: 0.1129 - mean_squared_error: 0.1129 - val_loss: 0.1369 - val_mean_squared_error: 0.1092

Epoch 00003: val_mean_squared_error improved from 0.12948 to 0.10921, saving model to combined0.h5
Epoch 4/50
300/300 [==============================] - 40s 134ms/step - loss: 0.0989 - mean_squared_error: 0.0989 - val_loss: 0.0580 - val_mean_squared_error: 0.0954

Epoch 00004: val_mean_squared_error improve

In [31]:
import subprocess
def split_model(composite, model_folder):
    best_model = keras.models.load_model(composite)
    history_net = keras.models.Model(inputs=[best_model.get_layer('history_inp').input], outputs=[best_model.get_layer('gru').output])

    vector_history_inp = Input((best_model.get_layer('gru').output.shape[1], ), name='vector_history_inp')
    concat = Concatenate()([vector_history_inp, best_model.get_layer('move_inp').output, best_model.get_layer('hand_inp').output])
    hidden = best_model.get_layer('hidden')(concat)
    output = best_model.get_layer('output')(hidden)

    move_inp = best_model.get_layer('move_inp').input
    hand_inp = best_model.get_layer('hand_inp').input
    position_net = keras.models.Model(inputs=[vector_history_inp, move_inp, hand_inp], outputs=[output])

    history_net.save(str(model_folder / 'history.h5'))
    position_net.save(str(model_folder / 'position.h5'))

In [34]:
from pathlib import Path

def delete_dir(path):
    if not os.path.exists(path):
        return
    for file in path.iterdir():
        os.remove(file)
    path.rmdir()

for i in range(num_train):
    combined_file = 'combined' + str(i) + '.h5'
    model_folder_name = data_folder + '_model' + str(i)
    
    model_folder_path = Path('../models/', model_folder_name)
    delete_dir(model_folder_path)
    model_folder_path.mkdir()
    
    split_model(combined_file, model_folder_path)
    print(model_folder_name)
    subprocess.check_output(['gsutil', 'cp', '-r', '../models/' + model_folder_name + '/*', 'gs://landlord_ai/models/' + model_folder_name])

3_29_sim11_model0
3_29_sim11_model1
3_29_sim11_model2
3_29_sim11_model3
3_29_sim11_model4
